In [5]:
import pandas as pd

/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [8]:
print(method_files,groundtruth_files)

{'05': ['05_aid.csv', '05_rcd.csv', '05_hira.csv', '05_dvca.csv'], '08': ['08_rcd.csv', '08_aid.csv', '08_hira.csv', '08_dvca.csv'], '07': ['07_hira.csv', '07_aid.csv', '07_rcd.csv', '07_dvca.csv'], '03': ['03_rcd.csv', '03_hira.csv', '03_aid.csv', '03_dvca.csv'], '01': ['01_rcd.csv', '01_hira.csv', '01_aid.csv', '01_dvca.csv'], '12': ['12_dvca.csv', '12_hira.csv', '12_aid.csv', '12_rcd.csv'], '04': ['04_aid.csv', '04_dvca.csv', '04_rcd.csv', '04_hira.csv'], '09': ['09_rcd.csv', '09_dvca.csv', '09_aid.csv', '09_hira.csv'], '06': ['06_dvca.csv', '06_rcd.csv', '06_aid.csv', '06_hira.csv'], '11': ['11_dvca.csv', '11_aid.csv', '11_rcd.csv', '11_hira.csv'], '02': ['02_rcd.csv', '02_aid.csv', '02_dvca.csv', '02_hira.csv'], '10': ['10_rcd.csv', '10_aid.csv', '10_dvca.csv', '10_hira.csv']} {'10': ['10_groundtruth.csv'], '08': ['08_groundtruth.csv'], '12': ['12_groundtruth.csv'], '09': ['09_groundtruth.csv'], '04': ['04_groundtruth.csv'], '02': ['02_groundtruth.csv'], '11': ['11_groundtruth.csv

In [174]:
import os
import pandas as pd
import ast
def flatten_nested_list(x):
    # Check if the first element is a list and it's not empty
    if x and isinstance(x[0], list):
        return x[0]
    return x


def load_and_process_data(file_id, method, directory="/home/anonymous/ACsim/results/inject_issues"):
    """
    Load ground truth and prediction data from specified files, and split their contents for further processing.

    Parameters:
    - file_id: str, identifier for the specific result files
    - method: str, methodology applied in the prediction file
    - directory: str, base directory where the files are stored

    Returns:
    - true_labels_lists: list, list of lists containing the ground truth labels
    - pred_labels_lists: list, list of lists containing the predicted labels
    """
    # Construct file paths
    groundtruth_file = f"{file_id}_groundtruth.csv"
    prediction_file = f"{file_id}_{method}.csv"
    groundtruth_path = os.path.join(directory, groundtruth_file)
    prediction_path = os.path.join(directory, prediction_file)

    # Load data from CSV files
    data = pd.read_csv(groundtruth_path)
    predictions = pd.read_csv(prediction_path)

    # Process the 'groundtruth_1' and 'Roots' columns by splitting each string by tabs
    true_labels_lists = data['groundtruth_1'].apply(lambda x: x.split('\t'))
    pred_labels_lists = predictions['Roots'].apply(lambda x: x.split('\t'))
    # pred_labels_lists = predictions['Roots'].apply(flatten_nested_list)
    # true_labels_lists = data['groundtruth_1'].apply(flatten_nested_list)
    
    return true_labels_lists, pred_labels_lists

def calculate_metrics(true_labels_lists, pred_labels_lists):
    """
    Calculate the True Positives (TP), False Positives (FP), and False Negatives (FN)
    from lists of true and predicted labels.

    Parameters:
    - true_labels_lists: list of lists, where each sublist contains true labels for a sample
    - pred_labels_lists: list of lists, where each sublist contains predicted labels for a sample

    Returns:
    - TP: int, number of True Positives
    - FP: int, number of False Positives
    - FN: int, number of False Negatives
    """
    TP = FP = FN = 0

    # Iterate over pairs of true and predicted labels
    for true_labels, pred_labels in zip(true_labels_lists, pred_labels_lists):
        # pred_label_list = ast.literal_eval(pred_labels) 
        # true_label_list = ast.literal_eval(true_labels)
        true_label_set = set(true_labels)
        # true_label_set = set(true_label_list)
        pred_label_set = set(pred_labels)
        # pred_label_set = set(pred_label_list)
        # Calculate metrics
        TP += len(true_label_set & pred_label_set)  # Intersection gives the TP
        FP += len(pred_label_set - true_label_set)  # Predicted labels not in true labels
        FN += len(true_label_set - pred_label_set)  # True labels not in predicted labels

    return TP, FP, FN

def calculate_evaluation(TP, FP, FN):
    """
    Calculate precision, recall, and F1-score.

    Args:
        TP (int): Number of True Positives
        FP (int): Number of False Positives
        FN (int): Number of False Negatives

    Returns:
        precision (float): The precision score.
        recall (float): The recall score.
        f1_score (float): The F1 score.
    """
    # Calculate precision
    if TP + FP > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    
    # Calculate recall
    if TP + FN > 0:
        recall = TP / (TP + FN)
    else:
        recall = 0

    # Calculate F1-score
    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0

        # Round and convert to percentage
    precision = round(precision * 100, 2)
    recall = round(recall * 100, 2)
    f1_score = round(f1_score * 100, 2)

    return precision, recall, f1_score
# Example usage:
file_id = "14"
method = "dvca"
true_labels, pred_labels = load_and_process_data(file_id, method)
# Example usage, assuming true_labels_lists and pred_labels_lists are already defined
TP, FP, FN = calculate_metrics(true_labels, pred_labels)
precision, recall, f1_score = calculate_evaluation(TP, FP, FN)
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print("Precision:", precision, "%")
print("Recall:", recall, "%")
print("F1 Score:", f1_score, "%")

True Positives (TP): 91
False Positives (FP): 9
False Negatives (FN): 109
Precision: 91.0 %
Recall: 45.5 %
F1 Score: 60.67 %


In [124]:
import os
import pandas as pd
import ast
def flatten_nested_list(x):
    # Check if the first element is a list and it's not empty
    if x and isinstance(x[0], list):
        return x[0]
    return x


def load_and_process_data(file_id, method, directory="/home/anonymous/ACsim/results/github_issues"):
    """
    Load ground truth and prediction data from specified files, and split their contents for further processing.

    Parameters:
    - file_id: str, identifier for the specific result files
    - method: str, methodology applied in the prediction file
    - directory: str, base directory where the files are stored

    Returns:
    - true_labels_lists: list, list of lists containing the ground truth labels
    - pred_labels_lists: list, list of lists containing the predicted labels
    """
    # Construct file paths
    groundtruth_file = f"{file_id}_groundtruth.csv"
    prediction_file = f"{file_id}_{method}.csv"
    groundtruth_path = os.path.join(directory, groundtruth_file)
    prediction_path = os.path.join(directory, prediction_file)

    # Load data from CSV files
    data = pd.read_csv(groundtruth_path)
    print(data.columns)
    data['groundtruth_2'] = data['groundtruth_2'].fillna('') 
    predictions = pd.read_csv(prediction_path)
    predictions['Roots'] = predictions['Roots'].fillna('') 
    
    # Process the 'groundtruth_1' and 'Roots' columns by splitting each string by tabs
    # true_labels_lists = data['groundtruth_1'].apply(lambda x: x.split('\t'))
    
    # pred_labels_lists = predictions['Roots'].apply(lambda x: x.split('\t'))
    true_labels_lists = data['groundtruth_1'].apply(flatten_nested_list)
    pred_labels_lists = predictions['Roots'].apply(flatten_nested_list)
    
    return true_labels_lists, pred_labels_lists

def calculate_metrics(true_labels_lists, pred_labels_lists):
    """
    Calculate the True Positives (TP), False Positives (FP), and False Negatives (FN)
    from lists of true and predicted labels.

    Parameters:
    - true_labels_lists: list of lists, where each sublist contains true labels for a sample
    - pred_labels_lists: list of lists, where each sublist contains predicted labels for a sample

    Returns:
    - TP: int, number of True Positives
    - FP: int, number of False Positives
    - FN: int, number of False Negatives
    """
    TP = FP = FN = 0

    # Iterate over pairs of true and predicted labels
    for true_labels, pred_labels in zip(true_labels_lists, pred_labels_lists):
        pred_label_list = ast.literal_eval(pred_labels)
        true_label_list = ast.literal_eval(true_labels)
        true_label_set = set(true_label_list)
        pred_label_set = set(pred_label_list)
        
        # Calculate metrics
        TP += len(true_label_set & pred_label_set)  # Intersection gives the TP
        FP += len(pred_label_set - true_label_set)  # Predicted labels not in true labels
        FN += len(true_label_set - pred_label_set)  # True labels not in predicted labels

    return TP, FP, FN

def calculate_evaluation(TP, FP, FN):
    """
    Calculate precision, recall, and F1-score.

    Args:
        TP (int): Number of True Positives
        FP (int): Number of False Positives
        FN (int): Number of False Negatives

    Returns:
        precision (float): The precision score.
        recall (float): The recall score.
        f1_score (float): The F1 score.
    """
    # Calculate precision
    if TP + FP > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    
    # Calculate recall
    if TP + FN > 0:
        recall = TP / (TP + FN)
    else:
        recall = 0

    # Calculate F1-score
    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0

        # Round and convert to percentage
    precision = round(precision * 100, 2)
    recall = round(recall * 100, 2)
    f1_score = round(f1_score * 100, 2)

    return precision, recall, f1_score
# Example usage:
file_id = "02"
method = "dvca"
true_labels, pred_labels = load_and_process_data(file_id, method)
print(true_labels,pred_labels)
# Example usage, assuming true_labels_lists and pred_labels_lists are already defined
TP, FP, FN = calculate_metrics(true_labels, pred_labels)
precision, recall, f1_score = calculate_evaluation(TP, FP, FN)
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print("Precision:", precision, "%")
print("Recall:", recall, "%")
print("F1 Score:", f1_score, "%")

Index(['groundtruth_1', 'groundtruth_2'], dtype='object')
0     ['euclidean_cluster']
1     ['euclidean_cluster']
2     ['euclidean_cluster']
3     ['euclidean_cluster']
4     ['euclidean_cluster']
              ...          
95    ['euclidean_cluster']
96    ['euclidean_cluster']
97    ['euclidean_cluster']
98    ['euclidean_cluster']
99    ['euclidean_cluster']
Name: groundtruth_1, Length: 100, dtype: object 0             ['object_association_merger_1']
1     ['obstacle_pointcloud_based_validator']
2     ['obstacle_pointcloud_based_validator']
3             ['object_association_merger_1']
4             ['object_association_merger_1']
                       ...                   
95            ['object_association_merger_1']
96            ['object_association_merger_1']
97    ['obstacle_pointcloud_based_validator']
98            ['clustering_shape_estimation']
99                      ['euclidean_cluster']
Name: Roots, Length: 100, dtype: object
True Positives (TP): 48
False Positives 

In [118]:
import os
import pandas as pd
import ast
def flatten_nested_list(x):
    # Check if the first element is a list and it's not empty
    if x and isinstance(x[0], list):
        return x[0]
    return x

def load_and_process_data(file_id, method, directory="/home/anonymous/ACsim/results/github_issues"):
    """
    Load ground truth and prediction data from specified files, and calculate the union of labels from groundtruth columns.

    Parameters:
    - file_id: str, identifier for the specific result files
    - method: str, methodology applied in the prediction file
    - directory: str, base directory where the files are stored

    Returns:
    - true_labels_union: list, list of lists containing the union of ground truth labels from multiple columns
    - pred_labels_lists: list, list of lists containing the predicted labels
    """
    # Construct file paths
    groundtruth_file = f"{file_id}_groundtruth.csv"
    prediction_file = f"{file_id}_{method}.csv"
    groundtruth_path = os.path.join(directory, groundtruth_file)
    prediction_path = os.path.join(directory, prediction_file)

    # Load data from CSV files
    data = pd.read_csv(groundtruth_path)
    predictions = pd.read_csv(prediction_path)

    # Ensure columns are filled with empty lists if NaN and evaluate strings to lists
    # for col in ['groundtruth_1', 'groundtruth_2', 'groundtruth_3']:
    for col in ['groundtruth_1', 'groundtruth_2']:
        data[col] = data[col].fillna('[]').apply(eval)

    # Calculate the union of the groundtruth columns
    def union_lists(row):
        # return list(set(row['groundtruth_1']) | set(row['groundtruth_2']) |set(row['groundtruth_3']))
        return list(set(row['groundtruth_1']) | set(row['groundtruth_2']))

    true_labels_union = data.apply(union_lists, axis=1)     # Process prediction labels if necessary
    predictions['Roots'] = predictions['Roots'].fillna('[]').apply(eval)
    pred_labels_lists = predictions['Roots'].apply(flatten_nested_list)
    return true_labels_union, pred_labels_lists

def calculate_metrics(true_labels_lists, pred_labels_lists):
    """
    Calculate the True Positives (TP), False Positives (FP), and False Negatives (FN)
    from lists of true and predicted labels.

    Parameters:
    - true_labels_lists: list of lists, where each sublist contains true labels for a sample
    - pred_labels_lists: list of lists, where each sublist contains predicted labels for a sample

    Returns:
    - TP: int, number of True Positives
    - FP: int, number of False Positives
    - FN: int, number of False Negatives
    """
    TP = FP = FN = 0

    # Iterate over pairs of true and predicted labels
    for true_labels, pred_labels in zip(true_labels_lists, pred_labels_lists):
        # pred_label_list = ast.literal_eval(pred_labels)
        # pred_label_set = set(pred_label_list)
        # true_label_list = ast.literal_eval(true_labels)
        # true_label_set = set(true_label_list)
        pred_label_set = set(pred_labels)
        true_label_set = set(true_labels)

        # Calculate metrics
        TP += len(true_label_set & pred_label_set)  # Intersection gives the TP
        FP += len(pred_label_set - true_label_set)  # Predicted labels not in true labels
        FN += len(true_label_set - pred_label_set)  # True labels not in predicted labels

    return TP, FP, FN

def calculate_evaluation(TP, FP, FN):
    """
    Calculate precision, recall, and F1-score.

    Args:
        TP (int): Number of True Positives
        FP (int): Number of False Positives
        FN (int): Number of False Negatives

    Returns:
        precision (float): The precision score.
        recall (float): The recall score.
        f1_score (float): The F1 score.
    """
    # Calculate precision
    if TP + FP > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    
    # Calculate recall
    if TP + FN > 0:
        recall = TP / (TP + FN)
    else:
        recall = 0

    # Calculate F1-score
    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0

        # Round and convert to percentage
    precision = round(precision * 100, 2)
    recall = round(recall * 100, 2)
    f1_score = round(f1_score * 100, 2)

    return precision, recall, f1_score
# Example usage:
file_id = "02"
method = "dvca"
true_labels, pred_labels = load_and_process_data(file_id, method)
# Example usage, assuming true_labels_lists and pred_labels_lists are already defined
TP, FP, FN = calculate_metrics(true_labels, pred_labels)
precision, recall, f1_score = calculate_evaluation(TP, FP, FN)
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print("Precision:", precision, "%")
print("Recall:", recall, "%")
print("F1 Score:", f1_score, "%")

True Positives (TP): 56
False Positives (FP): 44
False Negatives (FN): 144
Precision: 56.0 %
Recall: 28.0 %
F1 Score: 37.33 %


In [113]:
import os
import pandas as pd
import ast
def flatten_nested_list(x):
    # Check if the first element is a list and it's not empty
    if x and isinstance(x[0], list):
        return x[0]
    return x


def load_and_process_data(file_id, method, directory="/home/anonymous/ACsim/results/github_issues"):
    """
    Load ground truth and prediction data from specified files, and split their contents for further processing.

    Parameters:
    - file_id: str, identifier for the specific result files
    - method: str, methodology applied in the prediction file
    - directory: str, base directory where the files are stored

    Returns:
    - true_labels_lists: list, list of lists containing the ground truth labels
    - pred_labels_lists: list, list of lists containing the predicted labels
    """
    # Construct file paths
    groundtruth_file = f"{file_id}_groundtruth.csv"
    prediction_file = f"{file_id}_{method}.csv"
    groundtruth_path = os.path.join(directory, groundtruth_file)
    prediction_path = os.path.join(directory, prediction_file)

    # Load data from CSV files
    data = pd.read_csv(groundtruth_path)
    predictions = pd.read_csv(prediction_path)
    print(predictions.columns)
    predictions['Roots'] = predictions['Roots'].fillna('') 
    # Process the 'groundtruth_1' and 'Roots' columns by splitting each string by tabs
    true_labels_lists_1 = data['groundtruth_1'].apply(flatten_nested_list)
    true_labels_lists_2 = data['groundtruth_2'].apply(flatten_nested_list)
    # true_labels_lists_3 = data['groundtruth_3'].apply(flatten_nested_list)
    pred_labels_lists = predictions['Roots'].apply(flatten_nested_list)
    
    return true_labels_lists_1, true_labels_lists_2,pred_labels_lists

def calculate_metrics(true_labels_lists_1,true_labels_lists_2,  pred_labels_lists):
    """
    Calculate the True Positives (TP), False Positives (FP), and False Negatives (FN)
    from lists of true and predicted labels.

    Parameters:
    - true_labels_lists: list of lists, where each sublist contains true labels for a sample
    - pred_labels_lists: list of lists, where each sublist contains predicted labels for a sample

    Returns:
    - TP: int, number of True Positives
    - FP: int, number of False Positives
    - FN: int, number of False Negatives
    """
    TP = FP = FN = 0

    # Iterate over pairs of true and predicted labels
    for true_labels_1, true_labels_2, pred_labels in zip(true_labels_lists_1,true_labels_lists_2, pred_labels_lists):
        pred_label_list = ast.literal_eval(pred_labels)
        pred_label_set = set(pred_label_list)
        true_label_list_1 = ast.literal_eval(true_labels_1)
        true_label_set_1 = set(true_label_list_1)
        true_label_list_2 = ast.literal_eval(true_labels_2)
        true_label_set_2 = set(true_label_list_2)
        # true_label_list_3 = ast.literal_eval(true_labels_3)
        # true_label_set_3 = set(true_label_list_3)
         
        selected_true_label_set = true_label_set_1 if len(true_label_set_1 & pred_label_set) > len(true_label_set_2 & pred_label_set) else true_label_set_2
        TP += len( selected_true_label_set & pred_label_set)  # Intersection with either true label set gives the TP
        FP += len(pred_label_set -selected_true_label_set)  # Predicted labels not in either true label set
        FN += len(selected_true_label_set - pred_label_set)

        #FN += len((true_label_set_1 - pred_label_set)&(true_label_set_2 - pred_label_set)) # True labels not in predicted labels
        
    return TP, FP, FN

def calculate_evaluation(TP, FP, FN):
    """
    Calculate precision, recall, and F1-score.

    Args:
        TP (int): Number of True Positives
        FP (int): Number of False Positives
        FN (int): Number of False Negatives

    Returns:
        precision (float): The precision score.
        recall (float): The recall score.
        f1_score (float): The F1 score.
    """
    # Calculate precision
    if TP + FP > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    
    # Calculate recall
    if TP + FN > 0:
        recall = TP / (TP + FN)
    else:
        recall = 0

    # Calculate F1-score
    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0

        # Round and convert to percentage
    precision = round(precision * 100, 2)
    recall = round(recall * 100, 2)
    f1_score = round(f1_score * 100, 2)

    return precision, recall, f1_score
# Example usage:
file_id = "02"
method = "aid"
true_labels_1,true_labels_2,  pred_labels = load_and_process_data(file_id, method)
# Example usage, assuming true_labels_lists and pred_labels_lists are already defined
TP, FP, FN = calculate_metrics(true_labels_1, true_labels_2, pred_labels)
precision, recall, f1_score = calculate_evaluation(TP, FP, FN)
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print("Precision:", precision, "%")
print("Recall:", recall, "%")
print("F1 Score:", f1_score, "%")

Index(['experiment_id', 'id', 'Roots'], dtype='object')
True Positives (TP): 87
False Positives (FP): 13
False Negatives (FN): 13
Precision: 87.0 %
Recall: 87.0 %
F1 Score: 87.0 %
